In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!nvidia-smi
!wget https://github.com/gnina/gnina/releases/download/v1.3/gnina
!chmod +x gnina
!./gnina --version

Thu Jan  9 10:44:20 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda --version
!conda install conda-forge::pdbfixer
!conda install conda-forge::nglview
!conda install conda-forge::openbabel
!conda install conda-forge::mdanalysis
!conda install conda-forge::rdkit
!conda install conda-forge::py3dmol



Streaming output truncated to the last 5000 lines.
    cached_property-1.5.2      |     pyha770c72_1          11 KB  conda-forge
    comm-0.2.2                 |     pyhd8ed1ab_1          12 KB  conda-forge
    debugpy-1.8.11             |  py310hf71b8c6_0         2.0 MB  conda-forge
    decorator-5.1.1            |     pyhd8ed1ab_1          14 KB  conda-forge
    defusedxml-0.7.1           |     pyhd8ed1ab_0          23 KB  conda-forge
    entrypoints-0.4            |     pyhd8ed1ab_1          11 KB  conda-forge
    exceptiongroup-1.2.2       |     pyhd8ed1ab_1          20 KB  conda-forge
    executing-2.1.0            |     pyhd8ed1ab_1          28 KB  conda-forge
    fqdn-1.5.1                 |     pyhd8ed1ab_1          16 KB  conda-forge
    h11-0.14.0                 |     pyhd8ed1ab_1          51 KB  conda-forge
    h2-4.1.0                   |     pyhd8ed1ab_1          51 KB  conda-forge
    hpack-4.0.0                |     pyhd8ed1ab_1          29 KB  conda-forge
    httpcore-

In [1]:
import warnings
from pathlib import Path
import subprocess
import urllib

import requests
from pathlib import Path

from pdbfixer import PDBFixer
from openmm.app import PDBFile
import os

import nglview as nv
from openbabel import pybel
import torch

from rdkit import Chem
from rdkit.Chem import AllChem

pdb_ids = ["4IAQ", "6G79"]
Path("pdb_files").mkdir(exist_ok=True)

# Download PDB files
for pdb_id in pdb_ids:
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    with open(f"pdb_files/{pdb_id}.pdb", "wb") as file:
        file.write(response.content)
    print(f"PDB file {pdb_id} downloaded.")

PDB file 4IAQ downloaded.
PDB file 6G79 downloaded.


In [2]:
### Extract receptors and ligands from pdbs

!grep ATOM pdb_files/4IAQ.pdb > pdb_files/4IAQ_prot.pdb
!grep 2GM pdb_files/4IAQ.pdb > pdb_files/2GM_lig.pdb

!grep ATOM pdb_files/6G79.pdb > pdb_files/6G79_prot.pdb
!grep EP5 pdb_files/6G79.pdb > pdb_files/EP5_lig.pdb

### QUESTIONS:
1. Align protein structure ? 4IAQ to 6G79 or 4IAQ to 4IAQ prepared?
2. PDBFixer? Highly reduces redocking accuracy

3. How to get protein and ligand?
### Option 1: Extract (grep) protein and ligand from original pdb
### Option 2: Remove ligand from original pdb using PDBFixer, use SMILES to code the ligand

In [3]:
### Prepare the protein using PDBFixer ###

def prepare_protein(input_pdb, output_pdb):

    fixer = PDBFixer(filename=input_pdb)
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(True)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    PDBFile.writeFile(fixer.topology, fixer.positions, open(output_pdb, 'w'))
    print(f"Protein {input_pdb} prepared and saved as {output_pdb}")


for pdb_id in pdb_ids:
    prepare_protein(f"pdb_files/{pdb_id}_prot.pdb", f"pdb_files/{pdb_id}_prepared.pdb")


Protein pdb_files/4IAQ_prot.pdb prepared and saved as pdb_files/4IAQ_prepared.pdb
Protein pdb_files/6G79_prot.pdb prepared and saved as pdb_files/6G79_prepared.pdb


In [4]:
### Prepare ligands from SMILES ###

def prepare_ligand_from_smiles(smiles, output_sdf):
    try:
        mol = Chem.MolFromSmiles(smiles)  # Parse SMILES
        if mol is None:
            raise ValueError("Invalid SMILES string.")
        mol = Chem.AddHs(mol)  # Add explicit hydrogens
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Generate 3D coordinates
        Chem.MolToMolFile(mol, output_sdf)  # Save to SDF
        print(f"Ligand saved to: {output_sdf}")
    except Exception as e:
        print(f"Error during ligand preparation: {e}")

# SMILES string
smiles_2GM = "C[C@@]1(C(=O)N2[C@H](C(=O)N3CCC[C@H]3[C@@]2(O1)O)Cc4ccccc4)NC(=O)[C@@H]5C[C@@H]6c7cccc8c7c(c[nH]8)C[C@H]6N(C5)C"

# Prepare ligand
prepare_ligand_from_smiles(smiles_2GM, "./pdb_files/2GM.sdf")


# SMILES string
smiles_EP5 = "c1cc(ccc1C#N)N2CCN(CC2)C(=O)COc3ccc4c(c3)c(c[nH]4)CC[NH3+]"

# Prepare ligand
prepare_ligand_from_smiles(smiles_EP5, "./pdb_files/EP5.sdf")

Ligand saved to: ./pdb_files/2GM.sdf
Ligand saved to: ./pdb_files/EP5.sdf


In [7]:
!obabel pdb_files/2GM.sdf -O pdb_files/2GM_cleaned.sdf

!obabel pdb_files/EP5.sdf -O pdb_files/EP5_cleaned.sdf

1 molecule converted
1 molecule converted


In [9]:
### DOCK 2GM into 4IAQ (only grep without PDBFixer) ###

!./gnina -r pdb_files/4IAQ_prot.pdb \
         -l pdb_files/2GM_lig.pdb \
         --autobox_ligand pdb_files/2GM_lig.pdb \
         -o docking_output_4IAQ_2GM.sdf \
         --exhaustiveness 8 \
         --seed 123

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r pdb_files/4IAQ_prot.pdb -l pdb_files/2GM_lig.pdb --autobox_ligand pdb_files/2GM_lig.pdb -o docking_output_4IAQ_2GM.sdf --exhaustiveness 8 --seed 123
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

Using random seed: 123

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
 | pose 0 | ligand outside box
 | pose 0 | ligand outside box
 | pos

In [10]:
!obrms -f pdb_files/2GM_lig.pdb docking_output_4IAQ_2GM.sdf


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdb_files/2GM_lig.pdb)

RMSD pdb_files/2GM_lig.pdb: 0.160384
RMSD pdb_files/2GM_lig.pdb: 4.24227
RMSD pdb_files/2GM_lig.pdb: 2.06695
RMSD pdb_files/2GM_lig.pdb: 4.571
RMSD pdb_files/2GM_lig.pdb: 2.46557
RMSD pdb_files/2GM_lig.pdb: 4.07609
RMSD pdb_files/2GM_lig.pdb: 9.96055
RMSD pdb_files/2GM_lig.pdb: 2.16975
RMSD pdb_files/2GM_lig.pdb: 9.47773


In [11]:
### best mode
import py3Dmol

# Create 3Dmol.js viewer
v = py3Dmol.view()

# Load protein structure
v.addModel(open('pdb_files/4IAQ_prot.pdb').read())  # Protein
v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

# Load ligand structure
v.addModel(open('pdb_files/2GM_lig.pdb').read())  # Ligand
v.setStyle({'model': 1}, {'stick': {'colorscheme': 'dimgrayCarbon', 'radius': .125}})

# Extract only the first docking pose from the SDF file
with open('docking_output_4IAQ_2GM.sdf') as sdf_file:
    best_pose = []
    for line in sdf_file:
        if line.startswith('$$$$'):
            break  # End of the first molecule
        best_pose.append(line)

# Add the best docking pose to the viewer
v.addModel("".join(best_pose), 'sdf')  # Load only the best pose
v.setStyle({'model': 2}, {'stick': {'colorscheme': 'greenCarbon'}})

# Final adjustments
v.zoomTo({'model': 1})  # Focus on the protein
v.rotate(90)  # Rotate for better visualization
v.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [12]:
### DOCK 2GM into 4IAQ (PDBFixer) ###

!./gnina -r pdb_files/4IAQ_prepared.pdb \
         -l pdb_files/2GM_cleaned.sdf \
         --autobox_ligand pdb_files/2GM_lig.pdb \
         -o docking_output_4IAQ_2GM_fixer.sdf \
         --exhaustiveness 8 \
         --seed 123

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r pdb_files/4IAQ_prepared.pdb -l pdb_files/2GM_cleaned.sdf --autobox_ligand pdb_files/2GM_lig.pdb -o docking_output_4IAQ_2GM_fixer.sdf --exhaustiveness 8 --seed 123
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

Using random seed: 123

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
 | pose 0 | initial pose not within box
 | pose 0 | ligand outside box
 | pose 0 | ligand outside box

mode |  affinity  |  intramol  |    CNN     |   CNN
     | (kca

In [16]:
!obrms -f pdb_files/2GM_lig.pdb docking_output_4IAQ_2GM_fixer.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdb_files/2GM_lig.pdb)

RMSD pdb_files/2GM_lig.pdb: 0.997994
RMSD pdb_files/2GM_lig.pdb: 3.09137
RMSD pdb_files/2GM_lig.pdb: 3.06501
RMSD pdb_files/2GM_lig.pdb: 3.87274
RMSD pdb_files/2GM_lig.pdb: 9.58468
RMSD pdb_files/2GM_lig.pdb: 4.64053
RMSD pdb_files/2GM_lig.pdb: 9.55452
RMSD pdb_files/2GM_lig.pdb: 4.25896
RMSD pdb_files/2GM_lig.pdb: 4.11039


In [26]:
### best mode
import py3Dmol

# Create 3Dmol.js viewer
v = py3Dmol.view(width=1000, height=800)

# Load protein structure
v.addModel(open('pdb_files/4IAQ_prepared.pdb').read())  # Protein
v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

# Load ligand structure
v.addModel(open('pdb_files/2GM_lig.pdb').read())  # Ligand
v.setStyle({'model': 1}, {'stick': {'colorscheme': 'dimgrayCarbon', 'radius': .125}})

# Extract only the first docking pose from the SDF file
with open('docking_output_4IAQ_2GM_fixer.sdf') as sdf_file:
    best_pose = []
    for line in sdf_file:
        if line.startswith('$$$$'):
            break  # End of the first molecule
        best_pose.append(line)

# Add the best docking pose to the viewer
v.addModel("".join(best_pose), 'sdf')  # Load only the best pose
v.setStyle({'model': 2}, {'stick': {'colorscheme': 'greenCarbon'}})

# Final adjustments
v.zoomTo({'model': 1})  # Focus on the protein
v.rotate(90)  # Rotate for better visualization
v.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [17]:
### all modes

v = py3Dmol.view()
v.addModel(open('pdb_files/4IAQ_prot.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('pdb_files/2GM_lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('docking_output_4IAQ_2GM.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(90)
v.animate({'interval':3000})


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [25]:
### FLEXIBLY DOCK 2GM into 4IAQ (only grep without PDBFixer) ###

!./gnina -r pdb_files/4IAQ_prot.pdb \
         -l pdb_files/2GM_lig.pdb \
         --autobox_ligand pdb_files/2GM_lig.pdb \
         -o flex_docking_output_4IAQ_2GM.sdf \
         --flexdist 4 \
         --flexdist_ligand pdb_files/2GM_lig.pdb \
         --out_flex output/4IAQ_2GM_flexout.pdb \
         --exhaustiveness 8 \
         --seed 123



              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r pdb_files/4IAQ_prot.pdb -l pdb_files/2GM_lig.pdb --autobox_ligand pdb_files/2GM_lig.pdb -o flex_docking_output_4IAQ_2GM.sdf --flexdist 4 --flexdist_ligand pdb_files/2GM_lig.pdb --out_flex output/4IAQ_2GM_flexout.pdb --exhaustiveness 8 --seed 123
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

Flexible residues: A:125 A:126 A:129 A:130

In [20]:
!obrms -f pdb_files/2GM_lig.pdb flex_docking_output_4IAQ_2GM.sdf

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdb_files/2GM_lig.pdb)

RMSD pdb_files/2GM_lig.pdb: 0.160384
RMSD pdb_files/2GM_lig.pdb: 4.24227
RMSD pdb_files/2GM_lig.pdb: 2.06695
RMSD pdb_files/2GM_lig.pdb: 4.571
RMSD pdb_files/2GM_lig.pdb: 2.46557
RMSD pdb_files/2GM_lig.pdb: 4.07609
RMSD pdb_files/2GM_lig.pdb: 9.96055
RMSD pdb_files/2GM_lig.pdb: 2.16975
RMSD pdb_files/2GM_lig.pdb: 9.47773


In [22]:
### best mode
import py3Dmol

# Create 3Dmol.js viewer
v = py3Dmol.view()

# Load protein structure
v.addModel(open('pdb_files/4IAQ_prot.pdb').read())  # Protein
v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

# Load ligand structure
v.addModel(open('pdb_files/2GM_lig.pdb').read())  # Ligand
v.setStyle({'model': 1}, {'stick': {'colorscheme': 'dimgrayCarbon', 'radius': .125}})

# Extract only the first docking pose from the SDF file
with open('flex_docking_output_4IAQ_2GM.sdf') as sdf_file:
    best_pose = []
    for line in sdf_file:
        if line.startswith('$$$$'):
            break  # End of the first molecule
        best_pose.append(line)

# Add the best docking pose to the viewer
v.addModel("".join(best_pose), 'sdf')  # Load only the best pose
v.setStyle({'model': 2}, {'stick': {'colorscheme': 'greenCarbon'}})

# Final adjustments
v.zoomTo({'model': 1})  # Focus on the protein
v.rotate(90)  # Rotate for better visualization
v.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [23]:
### all modes

v = py3Dmol.view()
v.addModel(open('pdb_files/4IAQ_prot.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('pdb_files/2GM_lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('flex_docking_output_4IAQ_2GM.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(90)
v.animate({'interval':3000})


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
### DOCK EP5 into 6G79 ###

!./gnina -r pdb_files/6G79_receptor.pdb \
         -l pdb_files/EP5_ligand.pdb \
         --autobox_ligand pdb_files/EP5_ligand.pdb \
         -o docking_output_6G79_EP5.sdf \
         --exhaustiveness 8 \
         --seed 123

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r pdb_files/6G79_receptor.pdb -l pdb_files/EP5_ligand.pdb --autobox_ligand pdb_files/EP5_ligand.pdb -o docking_output_6G79_EP5.sdf --exhaustiveness 8 --seed 123
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

Using random seed: 123

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |  affinity  |  intramol  |    CNN     |   CNN
     

In [ ]:
!obrms -f pdb_files/EP5_ligand.pdb docking_output_6G79_EP5.sdf

import py3Dmol
v = py3Dmol.view()
v.addModel(open('pdb_files/6G79_receptor.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('pdb_files/EP5_ligand.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('docking_output_6G79_EP5.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(90)
v.animate({'interval':2000})


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is pdb_files/EP5_ligand.pdb)

RMSD pdb_files/EP5_ligand.pdb: 1.49764
RMSD pdb_files/EP5_ligand.pdb: 1.11528
RMSD pdb_files/EP5_ligand.pdb: 2.39281
RMSD pdb_files/EP5_ligand.pdb: 2.20885
RMSD pdb_files/EP5_ligand.pdb: 6.10449
RMSD pdb_files/EP5_ligand.pdb: 5.72437
RMSD pdb_files/EP5_ligand.pdb: 11.4389
RMSD pdb_files/EP5_ligand.pdb: 2.437
RMSD pdb_files/EP5_ligand.pdb: 2.61488


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
### FEXIBLE DOCKING 2GM into 4IAQ ###

!./gnina -r pdb_files/4IAQ_prepared.pdb \
         -l pdb_files/2GM_cleaned.sdf \
         --autobox_ligand pdb_files/2GM_lig.pdb \
         -o output/flex_docking_output_4IAQ_2GM.sdf \
         --flexdist 4 \
         --flexdist_ligand pdb_files/2GM_lig.pdb \
         --out_flex output/4IAQ_2GM_flexout.pdb \
         --exhaustiveness 8 \
         --seed 123